# Create a Deep Learning VM on Google Cloud Platform

This project is deployed on Google Cloud Platform in order to take advantage of GPU-accelerated training of the Generator and Discriminator networks.


Using: 1x NVIDIA Tesla P100 GPU


Following these instructions:
https://github.com/sdrangan/introml/blob/master/GCP/getting_started.md

In [25]:
from __future__ import print_function

import os
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.multiprocessing

from PIL import Image
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seem for reproducibility
manualSeed = 990
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

torch.multiprocessing.set_sharing_strategy('file_system')

Random Seed:  990


In [26]:
print(torch.cuda.is_available())
print(torch.__version__)
device = ("cuda:0" if torch.cuda.is_available() else "cpu")
device

True
1.5.0


'cuda:0'

In [27]:
dataroot = "data/road_images"

workers = 2  # Number of workers for dataloader
batch_size = 24
image_size = 400  # Size of training images. All images resized to this size.
n_z = 400  # Size of z latent vector (i.e. size of generator input)
n_ch = 1
n_G = 100  # Size of feature maps in generator
n_D = 100  # Size of feature maps in discriminator

n_epochs = 300
G_lr = 0.0001
D_lr = 0.0004
beta1 = 0.5  # Beta1 hyperparam for Adam optimizers

In [28]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.Grayscale(num_output_channels=1),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         num_workers=workers)
# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

OSError: [Errno 12] Cannot allocate memory

# Maximization / Minimization Problem for GANs

A neural network G(z, θ₁) is used to model the Generator mentioned above. It’s role is mapping input noise variables z to the desired data space x (say images). Conversely, a second neural network D(x, θ₂) models the discriminator and outputs the probability that the data came from the real dataset, in the range (0,1). In both cases, θᵢ represents the weights or parameters that define each neural network.

As a result, the Discriminator is trained to correctly classify the input data as either real or fake. This means it’s weights are updated as to maximize the probability that any real data input x is classified as belonging to the real dataset, while minimizing the probability that any fake image is classified as belonging to the real dataset. In more technical terms, the loss/error function used maximizes the function D(x), and it also minimizes D(G(z)).

Furthermore, the Generator is trained to fool the Discriminator by generating data as realistic as possible, which means that the Generator’s weight’s are optimized to maximize the probability that any fake image is classified as belonging to the real datase. Formally this means that the loss/error function used for this network maximizes D(G(z)).

Source: https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f



[AG] To summarize, the "adversarial" optimization goals are:

- DISCRIMINATOR: Train the Discriminator such that D(x) is maximized, where x is a sample from the original training dataset, and to minimize D(G(z)), which is the probability of a generated sample being classified as authentic.
- GENERATOR: train the Generator such that D(G(z)), the probability P that a generated sample will be classified as an authentic sample, is maximized.

# Loss Function

The networks use Binary-Crossentropy loss function, mathematically described as:

<img src = "files/gan_loss_function.png">

In [29]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### Define Generator and Discriminator and Initialize

In [30]:
convTransp1 = nn.ConvTranspose2d(n_z, n_G * 32, 3, 1, bias=False).to(device)
convTransp2 = nn.ConvTranspose2d(n_G * 32, n_G * 16, 5, 2, bias=False).to(device)
convTransp3 = nn.ConvTranspose2d(n_G * 16, n_G * 8, 5, 2, bias=False).to(device)
convTransp4 = nn.ConvTranspose2d(n_G * 8, n_G * 4, 5, 2, output_padding = 1, bias=False).to(device)
convTransp5 = nn.ConvTranspose2d(n_G * 4, n_G * 2, 5, 2, output_padding = 1, bias=False).to(device)
convTransp6 = nn.ConvTranspose2d(n_G * 2, n_G, 7, 2, output_padding = 1, bias=False).to(device)
convTransp7 = nn.ConvTranspose2d(n_G, n_ch, 6, 2, bias=False).to(device)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(n_z, n_G * 32, 3, 1, bias=False),
            nn.BatchNorm2d(n_G * 32),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G * 32, n_G * 16, 5, 2, bias=False),
            nn.BatchNorm2d(n_G * 16),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G * 16, n_G * 8, 5, 2, bias=False),
            nn.BatchNorm2d(n_G * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G * 8, n_G * 4, 5, 2, output_padding = 1, bias=False),
            nn.BatchNorm2d(n_G * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G * 4, n_G * 2, 5, 2, output_padding = 1, bias=False),
            nn.BatchNorm2d(n_G * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G * 2, n_G, 7, 2, output_padding = 1, bias=False),
            nn.BatchNorm2d(n_G),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_G, n_ch, 6, 2, bias=False),
#             nn.BatchNorm2d(n_ch),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [31]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(n_ch, n_D, 7, 2, dilation = 3, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_D, n_D*2, 5, 2, dilation = 3, bias=False),
            nn.BatchNorm2d(n_D * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_D*2, n_D*4, 5, 2, dilation = 3, bias=False),
            nn.BatchNorm2d(n_D * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
#             nn.MaxPool2d(2).to(device),
            
            nn.Conv2d(n_D*4, n_D*8, 5, 2, dilation = 3, bias=False),
            nn.BatchNorm2d(n_D * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_D*8, n_D*16, 3, 2, 0, dilation = 3, bias=False),
            nn.BatchNorm2d(n_D * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_D*16, 1, 4, 1, 0, bias=False),
#             nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [32]:
generator = Generator().to(device)
generator.apply(weights_init)  # Randomly initialize all weights to mean=0, stdev=0.2.
print(generator)

discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
print(discriminator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(400, 3200, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(3200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(3200, 1600, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(1600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(1600, 800, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (7): BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(800, 400, kernel_size=(5, 5), stride=(2, 2), output_padding=(1, 1), bias=False)
    (10): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(400, 200, kernel_size=(5, 5), stride=(2, 2), output_padding=(1, 1), bias=False)
    (13): BatchNorm2d(200, eps

# Define Loss Function, Set Label Values, and Create Optimizers

In [33]:
# Create batch of latent vectors that we will use to visualize the progression of the generator
fixed_noise = torch.randn(batch_size, n_z, 1, 1, device=device)

# Label values
y_real = 0.005
y_fake = 0.9

criterion = nn.MSELoss()
optim_D = optim.Adam(discriminator.parameters(), lr=D_lr, betas=(beta1, 0.999))
optim_G = optim.Adam(generator.parameters(), lr=G_lr, betas=(beta1, 0.999))

In [34]:
print(len(dataloader))

147


# Main Training Loop

In [35]:
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

In [36]:
print("Starting Training Loop...")
# For each epoch
for epoch in range(n_epochs):
    # For each batch in the dataloader
    for data in tqdm(dataloader):        
        # Train D with real batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), y_real, device=device)

        output = discriminator(real_cpu).view(-1)
        print("real images output: ", output)
        errD_real = criterion(output, label)
        discriminator.zero_grad()
        errD_real.backward()
        D_x = output.mean().item()

        ## Train D with fake batch
        noise = torch.randn(b_size, n_z, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(y_fake)
        
        output = discriminator(fake.detach()).view(-1)
        print("fake images output: ", output)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optim_D.step()
        
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake

        ## (2) Update G network: maximize log(D(G(z)))
        generator.zero_grad()
        label.fill_(y_real)  # fake labels are real for generator cost
        output = discriminator(fake).view(-1)
        
        errG = criterion(output, label)
        errG.backward()
        optim_G.step()
        
        D_G_z2 = output.mean().item()
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 1 == 0) or ((epoch == n_epochs-1)):
            with torch.no_grad():
                fake = generator(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1
    
    
    # Output training stats
    print('Epoch [%d/%d] \tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
          % (epoch, n_epochs, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))


  0%|          | 0/147 [00:00<?, ?it/s]

Starting Training Loop...
real images output:  tensor([0.5009, 0.5001, 0.5074, 0.5033, 0.4962, 0.4897, 0.4863, 0.4889, 0.4971,
        0.4975, 0.5231, 0.4926, 0.4986, 0.4996, 0.4937, 0.4912, 0.4957, 0.4852,
        0.5005, 0.5138, 0.5020, 0.4845, 0.4949, 0.4958], device='cuda:0',
       grad_fn=<ViewBackward>)
fake images output:  tensor([0.4874, 0.5077, 0.5009, 0.4987, 0.4879, 0.4953, 0.4935, 0.5002, 0.4994,
        0.4946, 0.4941, 0.4990, 0.5057, 0.5114, 0.4903, 0.4997, 0.5092, 0.5033,
        0.5161, 0.5080, 0.4998, 0.4872, 0.5046, 0.4864], device='cuda:0',
       grad_fn=<ViewBackward>)


  1%|          | 1/147 [00:02<06:54,  2.84s/it]

real images output:  tensor([0.3420, 0.3109, 0.3304, 0.3677, 0.2116, 0.3286, 0.3103, 0.4067, 0.2695,
        0.2839, 0.3304, 0.3283, 0.2787, 0.2713, 0.3305, 0.2731, 0.3070, 0.4016,
        0.3655, 0.3290, 0.3399, 0.3349, 0.2016, 0.3948], device='cuda:0',
       grad_fn=<ViewBackward>)
fake images output:  tensor([0.5352, 0.5304, 0.5273, 0.5643, 0.5375, 0.5121, 0.5392, 0.5415, 0.5421,
        0.5284, 0.5365, 0.5320, 0.5325, 0.5258, 0.5416, 0.5181, 0.5249, 0.5358,
        0.5481, 0.5299, 0.5377, 0.5489, 0.5163, 0.5138], device='cuda:0',
       grad_fn=<ViewBackward>)


  1%|▏         | 2/147 [00:05<06:30,  2.70s/it]

...

real images output:  tensor([0.2358, 0.1189, 0.3057, 0.2716, 0.2453, 0.2517, 0.0825, 0.3675, 0.2509,
        0.3122, 0.2750, 0.1928, 0.2634, 0.1268, 0.3405, 0.3024, 0.1189, 0.1229,
        0.2222, 0.2492, 0.2968, 0.2500, 0.1127, 0.2076], device='cuda:0',
       grad_fn=<ViewBackward>)
fake images output:  tensor([0.4780, 0.4794, 0.4620, 0.4754, 0.4868, 0.4811, 0.4914, 0.4932, 0.4806,
        0.4868, 0.4871, 0.4752, 0.4969, 0.4806, 0.4858, 0.4929, 0.4705, 0.4832,
        0.4740, 0.4803, 0.4726, 0.5035, 0.4996, 0.4768], device='cuda:0',
       grad_fn=<ViewBackward>)


  0%|          | 0/147 [00:00<?, ?it/s]

Epoch [0/300] 	Loss_D: 0.2306	Loss_G: 0.8306	D(x): 0.2301	D(G(z)): 0.4831 / 0.9163


  0%|          | 0/147 [00:00<?, ?it/s]


OSError: [Errno 12] Cannot allocate memory

# Visualizing Loss

We plot Loss for the Generator and Discriminator networks over # of cumulative training iterations.

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000/12, repeat_delay=1000/12, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()